In [ ]:
%pip install seaborn numpy matplotlib pandas

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3

# Put your own path here
db_path = '/your/azure/synapse/path/database.sqlite'

# Take different aspects of the database
with sqlite3.connect(db_path) as con:
    countries = pd.read_sql_query("SELECT * from Country", con)
    matches = pd.read_sql_query("SELECT * from Match", con)
    leagues = pd.read_sql_query("SELECT * from League", con)
    teams = pd.read_sql_query("SELECT * from Team", con)
    Player_detail = pd.read_sql_query("SELECT * from Player_Attributes", con)
    Player = pd.read_sql_query("SELECT * from Player", con)
    
# Setting default Seaborn theme and setting maximum number of rows displayed
sns.set()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# Shape the dataframe
Player_detail.shape
# Display the first five rows of the Player_detail DataFrame
Player_detail.head(5)

In [ ]:
# Create a 2x2 grid of subplots
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(20,14))

# Extract subsets of the Player_detail DataFrame based on certain conditions
attack_work_rate = Player_detail[Player_detail['attacking_work_rate'].isin(['medium','high','low'])]
defence_work_rate = Player_detail[Player_detail['defensive_work_rate'].isin(['medium','high','low'])]


# Plot a histogram of attacking work rates on the top-left subplot
sns.histplot(attack_work_rate['attacking_work_rate'], ax=axes[0, 0], color='green')
axes[0, 0].set_title('Distribution of Attacking Work Rates')
axes[0, 0].set_xlabel('Attacking Work Rate')
axes[0, 0].set_ylabel('Count')

# Plot a histogram of defensive work rates on the top-right subplot
sns.histplot(defence_work_rate['defensive_work_rate'], ax=axes[0, 1])
axes[0, 1].set_title('Distribution of Defensive Work Rates')
axes[0, 1].set_xlabel('Defensive Work Rate')
axes[0, 1].set_ylabel('Count')

# Plot a histogram of overall ratings on the bottom-left subplot
sns.histplot(Player_detail['overall_rating'], ax=axes[1, 0], color='red')
axes[1, 0].set_title('Distribution of Overall Ratings')
axes[1, 0].set_xlabel('Overall Rating')
axes[1, 0].set_ylabel('Count')

# Plot a histogram of potentials on the bottom-right subplot
sns.histplot(Player_detail['potential'], ax=axes[1, 1], color='orange')
axes[1, 1].set_title('Distribution of Potentials')
axes[1, 1].set_xlabel('Potential')
axes[1, 1].set_ylabel('Count')


In [ ]:
# Display the first five rows of the Player DataFrame
Player.head(5)
# Combine the Player_detail and Player
Players = pd.merge(Player_detail,Player,on='player_api_id',how='left')
# Print the dataframes shapes
print(Player.shape)
print(Player_detail.shape)
print(Players.shape)

In [ ]:
# Extract player data for analysis, focusing on the most recent Ballon d'Or winners and two of my favourite players(Suarez and Bale)
ballondr = Players[Players['player_name'].isin(['Lionel Messi','Cristiano Ronaldo','Luis Suarez','Gareth Bale'])]
ballondr.head(4)

In [ ]:
# Extract the mean overall ratings of the specified players
overall = ballondr[['overall_rating','player_name']].groupby('player_name').mean()
overall
# Lionel Messi and Cristiano are considered as a two of the best players from this era, Bale and Suares are also praised highly

In [ ]:
# Checking the overall mean of the all players 
Players['overall_rating'].describe()
# In this case the average overall rating is 68.6

In [ ]:
from datetime import date
# 
# Assuming 'date' and 'birthday' columns are present in the Players DataFrame
wales = Players[Players['player_name'].isin(['Gareth Bale', 'Aaron Ramsey', 'Ben Davies', 'Joe Allen', 'Chris Gunter', 'Wayne Hennessey'])].copy()

# Convert 'date' and 'birthday' columns to datetime
wales[['date', 'birthday']] = wales[['date', 'birthday']].apply(pd.to_datetime)

# Calculate age and filter players older than 18
wales['age'] = (wales['date'] - wales['birthday']).dt.days // 365
wales = wales[wales['age'] > 18]

# Display the five first rows of the dataframe
wales.head(5)

In [ ]:
# Show average ratings, age and name of selected Welsh players
wales_rating = wales[['player_name','age','overall_rating']].groupby(['age','player_name']).mean().unstack()
wales_rating.columns = ['Gareth Bale', 'Aaron Ramsey', 'Ben Davies', 'Joe Allen', 'Chris Gunter', 'Wayne Hennessey']
wales_rating.head(7)

In [ ]:
# Plot the ratings of Welsh players over the age range 2008-2016
ax = wales_rating.plot(figsize=(16, 8), marker='o')
plt.title("Welsh players ratings in 2008-2016")
plt.xlabel("Age")
plt.ylabel("Rating") 


In [ ]:
# Select specific Welsh national team from the Players DataFrame
wales = Players[Players['player_name'].isin(['Gareth Bale', 'Aaron Ramsey', 'Ben Davies', 'Joe Allen', 'Chris Gunter', 'Wayne Hennessey', 'James Collins', 'Joe Allen', 'Andy King', 'Sam Wokes', 'Hal Robson-Kanu'])]

# Extract the maximum overall rating for each Welsh player and add the max column
wales_max_rating = wales[['player_name','overall_rating']].groupby('player_name').max()
wales_max_rating['max'] = 1

# Drop duplicate rows based on player name and merge the original DataFrame with maximum rating DataFrame
wales = pd.merge(wales,wales_max_rating,on=['player_name','overall_rating'],how='inner')
wales = wales.drop_duplicates(subset=['player_name']).reset_index(drop=True)
# Assign the country 'wales' to each Welsh player
wales['country'] = 'wales'

In [ ]:
# Select specific Spanish national team from the Players DataFrame
germany = Players[Players['player_name'].isin(['Julian Draxler', 'Sami Khedira', 'Miroslav Klose', 'Lukas Podolski', 'Mesut Oezil', 'Toni Kroos', 'Bastian Schweinsteiger', 'Philipp Lahm', 'Shkodran Mustafi', 'Jerome Boateng', 'Mats Hummels'])]

# Extract the maximum overall rating for each Spanish player and add the 'max' column
germany_max_rating = germany[['player_name','overall_rating']].groupby('player_name').max()
germany_max_rating['max'] = 1

# Drop duplicate rows based on player name and merge the original DataFrame with maximum rating DataFrame
germany = pd.merge(germany,germany_max_rating,on=['player_name','overall_rating'],how='inner')
germany = germany.drop_duplicates(subset=['player_name']).reset_index(drop=True)
# Assign the country 'germany' to each German player
germany['country'] = 'germany'

In [ ]:
# Select specific Spanish national team from the Players DataFrame
spain = Players[Players['player_name'].isin(['David Villa', 'Juan Mata', 'Jordi Alba', 'Cesc Fabregas', 'Sergio Ramos', 'Sergio Busquets', 'Xavi Hernandez', 'Andres Iniesta', 'David Silva', 'Fernando Torres', 'Gerard Pique'])]

# Extract the maximum overall rating for each Spanish player and add the 'max' column
spain_max_rating = spain[['player_name', 'overall_rating']].groupby('player_name').max()
spain_max_rating['max'] = 1

# Drop duplicate rows based on player name and merge the original DataFrame with maximum rating DataFrame
spain = pd.merge(spain, spain_max_rating, on=['player_name', 'overall_rating'], how='inner')
spain = spain.drop_duplicates(subset=['player_name']).reset_index(drop=True)

# Assign the country 'spain' to each Spanish player
spain['country'] = 'spain'

In [ ]:
# Concatenating DataFrames
countries_player = pd.concat([wales,germany,spain],axis=0)

# Selecting players
selected_players = Players[Players['player_name'].isin(['David Villa', 'Juan Mata', 'Jordi Alba', 'Cesc Fabregas', 'Sergio Ramos', 'Sergio Busquets', 'Xavi Hernandez', 'Andres Iniesta', 'David Silva', 'Fernando Torres', 'Gerard Pique', 'Julian Draxler', 'Sami Khedira', 'Miroslav Klose', 'Lukas Podolski', 'Mesut Oezil', 'Toni Kroos', 'Bastian Schweinsteiger', 'Philipp Lahm', 'Shkodran Mustafi', 'Jerome Boateng', 'Mats Hummels', 'Gareth Bale', 'Aaron Ramsey', 'Ben Davies', 'Joe Allen', 'Chris Gunter', 'Wayne Hennessey', 'James Collins', 'Joe Allen', 'Andy King', 'Sam Wokes', 'Hal Robson-Kanu'])]

# Create a copy of the selected_players DataFrame
selected_players_copy = selected_players.copy()

# Convert 'date' and 'birthday' columns to datetime using .loc
selected_players_copy[['date', 'birthday']] = selected_players_copy[['date', 'birthday']].apply(pd.to_datetime)

# Calculate age and filter players older than 18 using .loc
selected_players_copy['age'] = (selected_players_copy['date'] - selected_players_copy['birthday']).dt.days // 365
selected_players_copy = selected_players_copy[selected_players_copy['age'] > 18]

In [ ]:
# Show average ratings, age, and name of selected players from multiple countries
selected_players_rating = selected_players_copy[['player_name', 'age', 'overall_rating']].groupby(['age', 'player_name']).mean().unstack()

selected_players_rating.columns = ['David Villa', 'Juan Mata', 'Jordi Alba', 'Cesc Fabregas', 'Sergio Ramos', 'Sergio Busquets', 'Xavi Hernandez', 'Andres Iniesta', 'David Silva', 'Fernando Torres', 'Gerard Pique', 'Julian Draxler', 'Sami Khedira', 'Miroslav Klose', 'Lukas Podolski', 'Mesut Oezil', 'Toni Kroos', 'Bastian Schweinsteiger', 'Philipp Lahm', 'Shkodran Mustafi', 'Jerome Boateng', 'Mats Hummels', 'Gareth Bale', 'Aaron Ramsey', 'Ben Davies', 'Joe Allen', 'Chris Gunter', 'Wayne Hennessey', 'James Collins', 'Joe Allen', 'Andy King', 'Sam Wokes', 'Hal Robson-Kanu']

selected_players_rating.head(10)

In [ ]:
# Plot the ratings of players from multiple countries over the age range 2008-2016
ax = selected_players_rating.plot(figsize=(16, 8), marker='o')
plt.title("Players Ratings Over Age (2008-2016) - Multiple Countries")
plt.xlabel("Age")
plt.ylabel("Rating")
plt.legend(title='Player Name', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# Concatenating DataFrames
countries_player = pd.concat([wales,germany,spain],axis=0)
# Selecting columns
countries_player = countries_player[['player_name','overall_rating', 'potential','crossing', 'finishing','heading_accuracy',
                                    'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
                                    'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
                                    'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
                                    'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
                                    'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle','height', 'weight']]
# Resetting index and setting player name as the new index
countries_player = countries_player.reset_index(drop=True)
countries_player = countries_player.set_index('overall_rating')
#Dropping player name column
countries_player = countries_player.drop(columns=['player_name'])
countries_player.shape

# Grouping the DataFrame by 'overall_rating' and calculating the mean for each group
average_stats_by_rating = countries_player_sorted.groupby('overall_rating').average()

# Sort the DataFrame by the 'overall_rating' column in descending order
average_stats_by_rating = countries_player.sort_values(by='overall_rating', ascending=False)

average_stats_by_rating.head(50)

In [ ]:
# Specify the path where you want to save the CSV file
csv_path = '/your/azure/synapse/path/soccerdata.csv'

# Write the DataFrame to a CSV file
average_stats_by_rating.to_csv(csv_path, index=True, header=True)